In [ ]:
%config InlineBackend.figure_format = 'retina'
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
plt.rcParams['pdf.fonttype'] = 42

In [ ]:
# load genome and annotation

from src.DMS_Profile import Genome, Annotation

genome = Genome('/Users/leo/Documents/repos/lsStruct/data/genome/scer_chr_kan.FASTA')
annotation = Annotation('/Users/leo/Documents/repos/lsStruct/data/genome/scer_kan.gff')

## Figure 1b

In [ ]:
from src.DMS_Profile import PRO_Profile, pro_combine_profiles
from src.pro_utils import moving_average

In [ ]:
# load data

spt4_pro = pro_combine_profiles([PRO_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_pro/pro_spt4_1_mRNA_pro.pkl', 'spt4_R1', genome, norm=False), 
                                 PRO_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_pro/pro_spt4_2_mRNA_pro.pkl', 'spt4_R2', genome, norm=False), 
                                 PRO_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_pro/pro_spt4_3_mRNA_pro.pkl', 'spt4_R3', genome, norm=False)], 'spt4', genome, norm=True)

wild_pro = pro_combine_profiles([PRO_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_pro/pro_wild_1_mRNA_pro.pkl', 'wild_R1', genome, norm=False), 
                                 PRO_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_pro/pro_wild_2_mRNA_pro.pkl', 'wild_R2', genome, norm=False),
                                 PRO_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_pro/pro_wild_3_mRNA_pro.pkl', 'wild_R3', genome, norm=False),
                                 PRO_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_pro/pro_wild_4_mRNA_pro.pkl', 'wild_R4', genome, norm=False), 
                                 PRO_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_pro/pro_wild_5_mRNA_pro.pkl', 'wild_R5', genome, norm=False),
                                 PRO_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_pro/pro_wild_6_mRNA_pro.pkl', 'wild_R6', genome, norm=False),
                                 PRO_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_pro/pro_wild_7_mRNA_pro.pkl', 'wild_R7', genome, norm=False), 
                                 PRO_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_pro/pro_wild_8_mRNA_pro.pkl', 'wild_R8', genome, norm=False)], 'wild', genome, norm=True)

In [ ]:
# plot individual profile

fig, axs = plt.subplots(2, 1, figsize=[6, 2], sharex=True, sharey=True)
plt.subplots_adjust(hspace=0.3)

avg_win = 5

# RPS17A
chrom = 12
coords = (225889, 226697)

wild_pro.plot_profile(chrom, coords[0], coords[1], genome, axs[0], avg_win=avg_win)
spt4_pro.plot_profile(chrom, coords[0], coords[1], genome, axs[1], avg_win=avg_win)

plt.setp(axs, ylim=[0, 4])
plt.xlabel('Genomic coordinate')
plt.ylabel('Pol II per million')
plt.show()

In [ ]:
# plot meta profile

avg_win = 8 # should be even
n_bins  = 100
surr    = 0.1
min_ppm = 1

plt.figure(figsize=(3, 2))

meta_wild, norm_x, bin_size = wild_pro.get_meta_profile_binned(annotation, n_bins=n_bins, surr=surr)
meta_spt4, norm_x, bin_size = spt4_pro.get_meta_profile_binned(annotation, n_bins=n_bins, surr=surr)

mask_wild = np.array([np.all(i == i) for i in meta_wild])
mask_spt4 = np.array([np.all(i == i) for i in meta_spt4])

meta_wild_mean = np.mean(meta_wild[mask_wild & mask_spt4], axis=0)
meta_spt4_mean = np.mean(meta_spt4[mask_wild & mask_spt4], axis=0)

plt.plot(norm_x, meta_wild_mean, 'or', ms=5, mew=0)
plt.plot(norm_x, meta_spt4_mean, 'og', ms=5, mew=0)

plt.plot(norm_x[int(avg_win/2):-int(avg_win/2)+1], moving_average(meta_wild_mean, avg_win), '-r', lw=2, label=f"wildtype")
plt.plot(norm_x[int(avg_win/2):-int(avg_win/2)+1], moving_average(meta_spt4_mean, avg_win), '-g', lw=2, label=f"∆STP4")

plt.plot([surr, surr], [np.min(meta_spt4_mean), np.max(meta_spt4_mean)], '--k')
plt.plot([1-surr, 1-surr], [np.min(meta_spt4_mean), np.max(meta_spt4_mean)], '--k')

plt.legend()
plt.xlabel('normalized Pol II position in gene')
plt.ylabel('mean Pol II per million')
plt.show()

## Figure 1c

In [ ]:
from src.DMS_Profile import DMS_Profile, combine_profiles, gini_for_transcripts

In [ ]:
# load DMS data

dms_pro = combine_profiles([ DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_1_mRNA_agg.pkl', 'wild_R1', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_2_mRNA_agg.pkl', 'wild_R2', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_3_mRNA_agg.pkl', 'wild_R3', genome),
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_3_mRNA_agg.pkl', 'wild_R3', genome),
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_4_mRNA_agg.pkl', 'wild_R4', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_5_mRNA_agg.pkl', 'wild_R5', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_6_mRNA_agg.pkl', 'wild_R6', genome),
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_7_mRNA_agg.pkl', 'wild_R7', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_8_mRNA_agg.pkl', 'wild_R8', genome)], 'all_pro', genome, min_cov=900)

dms_mat = combine_profiles([ DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/mat_wild_1_mRNA_agg.pkl', 'wild_R1', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/mat_wild_2_mRNA_agg.pkl', 'wild_R2', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/mat_wild_3_mRNA_agg.pkl', 'wild_R3', genome)], 'all_mat', genome, min_cov=900)

In [ ]:
# plot individual profile for DBP2

chrom = 13
coords = (414760, 415000)
strand = '+'

fig, axs = plt.subplots(2, 1, figsize=[15, 2], sharex=True)
plt.subplots_adjust(hspace=0.1)

ax0 = axs[0].twinx()
dms_pro.plot_profile(chrom, strand, coords[0], coords[1], genome, axs[0], cmap_loc='../cmap.txt')
ax0.bar(np.arange(coords[0], coords[1]), dms_pro.cov_p[chrom][coords[0]:coords[1]], 1, color='lightgray')
axs[0].set_zorder(10)
axs[0].set_frame_on(False)

ax1 = axs[1].twinx()
dms_mat.plot_profile(chrom, strand, coords[0], coords[1], genome, axs[1], cmap_loc='../cmap.txt')
ax1.bar(np.arange(coords[0], coords[1]), dms_mat.cov_p[chrom][coords[0]:coords[1]], 1, color='lightgray')
axs[1].set_zorder(10)
axs[1].set_frame_on(False)
axs[1].set_ylabel('DMS Reactivity')
axs[1].set_xlabel('Genomic coordinate')

plt.setp([axs[0], axs[1]], ylim=(0, 2.5))
plt.show()

## Figure 1d

In [ ]:
from src.DMS_Profile import Targeted_DMS_Profile, targeted_combine_profiles
from src.dms_utils import sliding_corrcoeff, gini
from scipy.stats import linregress
from matplotlib.gridspec import GridSpec
from src.pydscatter import dscatter_plot
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as mplc

In [ ]:
def get_corr_vec(rea1, rea2, mask, w):
    # wrapper for sliding_corrcoeff() to handle NaNs
    mask_join = (rea1 == rea1) & (rea2 == rea2) & mask
    r = sliding_corrcoeff(rea1[mask_join], rea2[mask_join], w)
    _, _, r_value, _, _ = linregress(rea1[mask_join], rea2[mask_join])
    x = np.arange(1, len(mask)+1)[mask_join]
    return(x, r, r_value)

def gini_windows(rea, mask, w, min_n=20):
    ginis = []
    x = []
    for window in range(len(rea)-w):
        if len((rea[window:window+w])[mask[window:window+w]]) >= min_n:
            gini_w = gini((rea[window:window+w])[mask[window:window+w]])
            ginis.append(gini_w)
        else:
            ginis.append(np.nan)
        x.append((window+window+w)/2)
    return(x, ginis)

def get_partner(dbr):
    bpdict = {}
    opened = []
    for i, d in enumerate(dbr):
        if d == '(':
            opened.append(i)
        elif d == ')':
            partner = opened.pop(-1)
            if i not in bpdict:
                bpdict[i] = partner
                bpdict[partner] = i
    
    if opened:
        print('parentheses unmatched!')
        return(None)
    else:
        return(bpdict)

In [ ]:
# load nascent and mature average rRNA data
genome_rrna = Genome('/Users/leo/Documents/repos/lsStruct/data/genome/scer_chr_kan.FASTA', coords=(11, 451575, 458433), reverse=True)

wild_rib_1 = Targeted_DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/rib_wild_1_rRNA_agg.pkl', 'wild_R1', genome_rrna, reverse=True)
wild_rib_2 = Targeted_DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/rib_wild_2_rRNA_agg.pkl', 'wild_R2', genome_rrna, reverse=True) 
wild_rib_3 = Targeted_DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/rib_wild_3_rRNA_agg.pkl', 'wild_R3', genome_rrna, reverse=True)
wild_rib = targeted_combine_profiles([wild_rib_1, wild_rib_2, wild_rib_3], 'wild', genome_rrna, per_exclude=5)

wild_nas_1 = Targeted_DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_1_rRNA_agg.pkl', 'wild_R1', genome_rrna, reverse=True) 
wild_nas_2 = Targeted_DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_2_rRNA_agg.pkl', 'wild_R2', genome_rrna, reverse=True)
wild_nas_3 = Targeted_DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_3_rRNA_agg.pkl', 'wild_R3', genome_rrna, reverse=True)
wild_nas = targeted_combine_profiles([wild_nas_1, wild_nas_2, wild_nas_3], 'wild', genome_rrna, per_exclude=10)

In [ ]:
# ribosomal RNA secondary structure
dbr_18 = '...((((.........))))((((.(((((((.(((((((((.....(((.(((..((...(((..(.((..........)))..))))).....((((.......(((((((..((..(((((((............(((((....(((((((.....)))))))....)))))......(((((((((....)))))))))(((.(((((((.......(((((.(((....)))...))))).....))))))).)..))...((((.((((.....))))))))..))))))).))))))))).(((..(.(((....((((((((.......))))))))))).....))))...((((((((....))))...))))))))((((((..........)))))).((((....))))...)))))))......(.(((...(((((...))))).)))).)).))))))....((((..(((((((....)))))))..).))).....((((((((.......))))))))........((.((......(.((((((..(((....)))....))))))))).)).))))))))))).....(...(((.......((((...(((.((....((((((((((((.((((.(((.....)))...)))).....))))))))))))....((((((....(((((((((.....)))))))))....))))))..(((((((((.......(((..(.(...).)..(((.....)))...)))......)))))..)))).....(.((....(.((.(((.............))).))..)..)).)..))...((((((((((.((((((((((((((((((((...(((......)))......))))))))))))....(..((....)))))))))))))))).))))..))))...)))).(..((((((...(((.(((((.........))))).)))))))))..).......((((((.(((..(((((((...((..........)))))))))..)))...((....))...)))....))).))))(((((.((.((((....)))))))))))........(((((.(((((((..((..(((((((((((((((((.(.)((((........))))........(((((((....(((((....(((((((((..........)))))))))..))))).(.((.((((..((((((((((..(((((((((....)))..((((......))))..)))))).....((((((((.((((..(((((.((((((....))))))...)))))..))))))).((.(((((((...)))))))))....)))))...))))).)))...).))))))))....)))))))...)).)))))))))((..(((((((.(...(((.....(((.((((....)))).)))....)))....).)))))))....).((((((((((((........))))))))))))..).))))))(...(((((((((.......)))))))))..)..))...)))))))))).))....((.((...(((((((((((.((((((((((((..(((((((((((((((((((((((((((....)))))))))))))))))))))))))))..)))))))))))))))))))))))....))..))....((((((((((....))))))))))........'
dbr_58 = '........................................((((((((......)))).....((((((....................)))).))...))))...(((...)))(((((((((....))))))))).....................'
dbr_25 = '..........................((((((.....((....))......))))).).........((....))..(((((......((.....)).....))))).((((..((((((((((((((((((...))))))))))))))))))....(((((((.((((((((((((((((((((...(.(((((....))))))..(...).(((.......)))....))))))))..))))))))....)))))))))))........(((((((........))))))).....(((((........)))))..)))).................................((....))...............((((....))))..................................((((((((((((.(..(((((...((((..((((((((.....))))))))......))))...))))).(((((((((((((((((((((((......((((((((..((((((((((....))))))..)))).))))))))....))))).))))))))))).......(((....))).....)))))))..).))))))))))))..((((........))))((((((....(((((((((((((..((((..((((.....))))..))))...((....)).......(((((..(((((....)))))...))))).((((((((...((....))...))).))).))....)))))))))))))..((((...((((((...((((((((((((.((((...(((((((....)))))))....))))...((((.((.((........))))))))..).))))))))))).(((....))).......))))))......))))..((((((....(((((....))))).(((((((((.......((((((((..(.(((....((((((((((..((((((((.....))))))))))))))))))..)).).)..)))))))).(((((((((.........)))))))))...........)))))))).)...((((((((((.....))))))))))....(((......)))...(((((((...(((......(.((((((.................(((.((((((((((.......(((((.(((..(((.........)))..)))(...)(...((......)).).)))))...)))))))))))))...................))))))))))...)))))))(((((((((.........)))))))))....))))))(.((((((((.((......((((.((((....))))))))...))))))))))..).....))))))((((.((((.....((((((.......))))))((((.....(((((.....(((((((...((.....)).))))))).....(((.((...(((((.(((....))))))......(((((........))))).....))....))...(((((........)))))..((((.(((((((.....(((....))).......((((((...((((.(((((((.(((((((((.........))))))...(((((((..(((((((((((((.....)))))))))))))..)).)))))......(((((((....)))))))..)))...))))))).)))).........))))))............))))))))))).))).........((......))))))).((((.....))))..))))..((((....((((((((((((..............(((((...((....))....)))))((((((((((((((.((((((((((((((((((....(((((((((((((((((((.(((((((((((((....))))))))))))).))))))))))))).))))))....))))))))))(((((.....)))))....))))))))......)))))..)))))))))(...).......(((((((((..(((.........)))..(((((((((..((((.....))))..)..))))))))..((((.(((..((((((.(..(((....(((((....)))))....))))))))..)))))((((((.......))))))(...).((((.(...(.)).(((((((.)..))))))..)..)))..))))...)).)))))))....)))))))))))).))))))))))))....(((((........)))))..((((((((.(((.((.....(((((.(((((((.(((((..(((((((..((((((((((((((((((.........(((((..((....))............))))))))))))).))))))))))...(((((.....((((..(((((......)))))...............((((((....))))))..))))..))))).....)))))))...)))))..((((.....))).)((((((...........))))))..((..((((((((((.((((((.......))))))...))((......))....))))))))..(((((((((.......))))))))).(((....))).))..........((((((..((((.......)))))))))).........)))))..))))))).......(((..(((((((...((.......))...)))))))...)))..........(((((((((((..((((((((((....))))))))..)).(((((.....))))).....)))))..)..)))).).....(((((((....))).))))....))..)))))))))))..((((((((..((((.((((((((...............))))))))(((((...(((....(((((((((((....))))))..)))))...)))..)))))..(((....(((((..........)))))....))).))))....)))))))).......(((((((((((((................((((((.....))))))...............((((((((..(((((((((((......))))))))))))))))))).............)))))))))))))..((((((((((.......)))))....(((((((((.((..(..((((.((((((((......)))))))))))))...))....))).))))))..)))))..'
dbr_full = 701*'?' + dbr_18 + 361*'?' + dbr_58 + 232*'?' + dbr_25 + 210*'?'
mask_dat = np.array([True if i in [')', '(', '.'] else False for i in dbr_full])

## calculate sliding correlation coefficients
mask_acu = genome_rrna.mask_A()|genome_rrna.mask_C()|genome_rrna.mask_U() # joint mask for mature and nascent RNA

w = 50
x_nasvmat, r_nasvmat, r2_nasvmat = get_corr_vec(wild_nas.rat, wild_rib.rat, mask_acu, w)
x_nas,     r_nas    , r2_nas     = get_corr_vec(wild_nas_1.rat, wild_nas_2.rat, mask_acu, w)
x_mat,     r_mat    , r2_mat = get_corr_vec(wild_rib_1.rat, wild_rib_2.rat, mask_acu, w)

## calculate sliding gini index
w = 80
x_gini_nas, gini_nas = gini_windows(wild_nas.rat, mask_acu&wild_nas.mask, w, min_n=50)
x_gini_mat, gini_mat = gini_windows(wild_rib.rat, mask_acu&wild_rib.mask, w, min_n=50)

In [ ]:
fig = plt.figure(layout="constrained", figsize=(13,4))

gs = GridSpec(4, 2, figure=fig, width_ratios=(0.88, 0.12))
ax1 = fig.add_subplot(gs[0, 0])
ax2 = fig.add_subplot(gs[1, 0], sharex=ax1)
ax3 = fig.add_subplot(gs[2, 0], sharex=ax1)
ax4 = fig.add_subplot(gs[0, 1])
ax5 = fig.add_subplot(gs[1, 1])
ax6 = fig.add_subplot(gs[2, 1])
ax7 = fig.add_subplot(gs[3, 0], sharex=ax1)
ax8 = fig.add_subplot(gs[3, 1])

cmap = np.array(pd.read_csv('../cmap.txt', header=None))/255
cmap = mplc.ListedColormap(cmap)

ax1.fill_between(np.arange(0, len(mask_dat)), mask_dat, color='lightgray')
ax2.fill_between(np.arange(0, len(mask_dat)), mask_dat, color='lightgray')
ax3.fill_between(np.arange(0, len(mask_dat)), mask_dat, color='lightgray')
ax1.plot(x_nasvmat, r_nasvmat, '.')
ax2.plot(x_nas, r_nas, '.')
ax2.set_ylabel("Pearson's correlation")
ax3.plot(x_mat, r_mat, '.')

dscatter_plot(np.log10(wild_rib.rat[mask_acu]), np.log10(wild_nas.rat[mask_acu]), ax=ax4, order=True, alpha=0.15, markersize=5, edgecolors='none', rasterized=True, cmap=cmap)
ax4.set_aspect('equal', adjustable="datalim")
ax4.set_xticks(np.arange(-6, 1, 1))
ax4.set_yticks(np.arange(-4, 1, 1))
ax4.text(0.005, 0.82, f"R$={r2_nasvmat:.3f}", transform=ax4.transAxes)
dscatter_plot(np.log10(wild_nas_1.rat[mask_acu]), np.log10(wild_nas_2.rat[mask_acu]), ax=ax5, order=True, alpha=0.15, markersize=5, edgecolors='none', rasterized=True, cmap=cmap)
ax5.set_aspect('equal', adjustable="datalim")
ax5.set_xticks(np.arange(-6, 1, 1))
ax5.set_yticks(np.arange(-4, 1, 1))
ax5.text(0.005, 0.82, f"R$={r2_nas:.3f}", transform=ax5.transAxes)
dscatter_plot(np.log10(wild_rib_1.rat[mask_acu]), np.log10(wild_rib_2.rat[mask_acu]), ax=ax6, order=True, alpha=0.15, markersize=5, edgecolors='none', rasterized=True, cmap=cmap)
ax6.set_aspect('equal', adjustable="datalim")
ax6.set_xticks(np.arange(-10, 2, 2))
ax6.set_yticks(np.arange(-8, 2, 2))
ax6.text(0.005, 0.82, f"R$={r2_mat:.3f}", transform=ax6.transAxes)

ax7.plot(x_gini_nas, gini_nas)
ax7.plot(x_gini_mat, gini_mat)
ax7.set_ylim([0.3, 1])
ax7.set_xlabel('Position in primary rRNA transcript')
ax7.set_ylabel('Gini index')
dscatter_plot(gini_mat, gini_nas, ax=ax8, order=True, alpha=0.15, markersize=5, edgecolors='none', rasterized=True, cmap=cmap)
ax8.plot([0, 1], [0,  1], '--k')
ax8.set_aspect('equal', adjustable="datalim")
ax8.set_xlim([0.3, 1])
ax8.set_ylim([0.3, 1])
ax8.set_xticks(np.arange(0.2, 1.1, 0.2))
ax8.set_yticks(np.arange(0.2, 1.1, 0.2))

plt.setp([ax1, ax2, ax3], ylim=[-0.05, 1.05])
plt.show()

## Figure S1a

In [ ]:
# load data
data = pd.read_csv('../data/RQ19739-RQ19825-cD1000_Electropherogram.csv')

# plot
srt = 350
end = 600

fig, axs = plt.subplots(1, 2, figsize=(8, 3))

axs[0].plot(data['A1: Ladder'][srt:end]/np.sum(data['A1: Ladder'][srt:end]), label='DNA ladder')
axs[1].plot(data['D3: Ladder'][srt:end]/np.sum(data['A1: Ladder'][srt:end]))

# mature
axs[0].plot(data['C2: mat_d3_1'][srt:end]/np.sum(data['C2: mat_d3_1'][srt:end]), '-k')
axs[0].plot(data['C1: mat_wt_2'][srt:end]/np.sum(data['C1: mat_wt_2'][srt:end]), '-k')
axs[0].plot(data['E2: mat_d3_3'][srt:end]/np.sum(data['E2: mat_d3_3'][srt:end]), '-k', label='libraries')
axs[0].set_title('DMS-MaPseq library examples')
axs[0].legend()

# nascent
axs[1].plot(data['C5: pro_s1_3'][srt:end]/np.sum(data['C5: pro_s1_3'][srt:end]), '-k')
axs[1].plot(data['D5: pro_d7_1'][srt:end]/np.sum(data['D5: pro_d7_1'][srt:end]), '-k')
axs[1].plot(data['E5: pro_d7_2'][srt:end]/np.sum(data['E5: pro_d7_2'][srt:end]), '-k')
axs[1].set_title('CoSTseq library examples')

plt.show()

## Figure S1b

In [ ]:
dms_pro = combine_profiles([ DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_1_mRNA_agg.pkl', 'wild_R1', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_2_mRNA_agg.pkl', 'wild_R2', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_wild_3_mRNA_agg.pkl', 'wild_R3', genome),
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_dbp7_1_mRNA_agg.pkl', 'dbp7_R1', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_dbp7_2_mRNA_agg.pkl', 'dbp7_R2', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_dbp7_3_mRNA_agg.pkl', 'dbp7_R3', genome),
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_dbp3_1_mRNA_agg.pkl', 'dbp3_R1', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_dbp3_2_mRNA_agg.pkl', 'dbp3_R2', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_dbp3_3_mRNA_agg.pkl', 'dbp3_R3', genome),
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_spt4_1_mRNA_agg.pkl', 'spt4_R1', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_spt4_2_mRNA_agg.pkl', 'spt4_R2', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_spt4_3_mRNA_agg.pkl', 'spt4_R3', genome),
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_stm1_1_mRNA_agg.pkl', 'stm1_R1', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_stm1_2_mRNA_agg.pkl', 'stm1_R2', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_stm1_3_mRNA_agg.pkl', 'stm1_R3', genome)], 'all_pro', genome, min_cov=900)

dms_mat = combine_profiles([ DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/mat_wild_1_mRNA_agg.pkl', 'wild_R1', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/mat_wild_2_mRNA_agg.pkl', 'wild_R2', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/mat_wild_3_mRNA_agg.pkl', 'wild_R3', genome),
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/mat_dbp7_1_mRNA_agg.pkl', 'dbp7_R1', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/mat_dbp7_2_mRNA_agg.pkl', 'dbp7_R2', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/mat_dbp7_3_mRNA_agg.pkl', 'dbp7_R3', genome),
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/mat_dbp3_1_mRNA_agg.pkl', 'dbp3_R1', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/mat_dbp3_2_mRNA_agg.pkl', 'dbp3_R2', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/mat_dbp3_3_mRNA_agg.pkl', 'dbp3_R3', genome),
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/mat_spt4_1_mRNA_agg.pkl', 'spt4_R1', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/mat_spt4_2_mRNA_agg.pkl', 'spt4_R2', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/mat_spt4_3_mRNA_agg.pkl', 'spt4_R3', genome),
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/mat_stm1_1_mRNA_agg.pkl', 'stm1_R1', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/mat_stm1_2_mRNA_agg.pkl', 'stm1_R2', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/mat_stm1_3_mRNA_agg.pkl', 'stm1_R3', genome)], 'all_mat', genome, min_cov=900)

unt_mat = combine_profiles([ DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/mat_unwt_1_mRNA_agg.pkl', 'wild_R1', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/mat_dms/mat_unwt_2_mRNA_agg.pkl', 'wild_R2', genome)], 'all_mat', genome, min_cov=900)
                           
unt_pro = combine_profiles([ DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_unwt_1_mRNA_agg.pkl', 'wild_R1', genome), 
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_unwt_2_mRNA_agg.pkl', 'wild_R2', genome),
                             DMS_Profile('/Users/leo/Documents/repos/PRODMSMaPseq/data/muts/pro_dms/pro_unwt_3_mRNA_agg.pkl', 'wild_R2', genome)], 'all_mat', genome, min_cov=900)


In [ ]:
# calculate mutation rates for individual nucleotides

def apply_mask(profile, mask):
    arr = np.hstack([p[msk] for p, msk in zip(profile, mask)])
    return(arr)

def comb_mask(mask1, mask2):
    return([i&j for i, j in zip(mask1, mask2)])

# individual nt masks
U = genome.mask_U()
A = genome.mask_A()
G = genome.mask_G()
C = genome.mask_C()
    
# treated
pro_U = np.hstack([apply_mask(dms_pro.rat_p, comb_mask(U, dms_pro.mask_p)), apply_mask(dms_pro.rat_m, comb_mask(A, dms_pro.mask_m))])
pro_A = np.hstack([apply_mask(dms_pro.rat_p, comb_mask(A, dms_pro.mask_p)), apply_mask(dms_pro.rat_m, comb_mask(U, dms_pro.mask_m))])
pro_G = np.hstack([apply_mask(dms_pro.rat_p, comb_mask(G, dms_pro.mask_p)), apply_mask(dms_pro.rat_m, comb_mask(C, dms_pro.mask_m))])
pro_C = np.hstack([apply_mask(dms_pro.rat_p, comb_mask(C, dms_pro.mask_p)), apply_mask(dms_pro.rat_m, comb_mask(G, dms_pro.mask_m))])

mat_U = np.hstack([apply_mask(dms_mat.rat_p, comb_mask(U, dms_mat.mask_p)), apply_mask(dms_mat.rat_m, comb_mask(A, dms_mat.mask_m))])
mat_A = np.hstack([apply_mask(dms_mat.rat_p, comb_mask(A, dms_mat.mask_p)), apply_mask(dms_mat.rat_m, comb_mask(U, dms_mat.mask_m))])
mat_G = np.hstack([apply_mask(dms_mat.rat_p, comb_mask(G, dms_mat.mask_p)), apply_mask(dms_mat.rat_m, comb_mask(C, dms_mat.mask_m))])
mat_C = np.hstack([apply_mask(dms_mat.rat_p, comb_mask(C, dms_mat.mask_p)), apply_mask(dms_mat.rat_m, comb_mask(G, dms_mat.mask_m))])

# untreated
unp_U = np.hstack([apply_mask(unt_pro.rat_p, comb_mask(U, unt_pro.mask_p)), apply_mask(unt_pro.rat_m, comb_mask(A, unt_pro.mask_m))])
unp_A = np.hstack([apply_mask(unt_pro.rat_p, comb_mask(A, unt_pro.mask_p)), apply_mask(unt_pro.rat_m, comb_mask(U, unt_pro.mask_m))])
unp_G = np.hstack([apply_mask(unt_pro.rat_p, comb_mask(G, unt_pro.mask_p)), apply_mask(unt_pro.rat_m, comb_mask(C, unt_pro.mask_m))])
unp_C = np.hstack([apply_mask(unt_pro.rat_p, comb_mask(C, unt_pro.mask_p)), apply_mask(unt_pro.rat_m, comb_mask(G, unt_pro.mask_m))])

unm_U = np.hstack([apply_mask(unt_mat.rat_p, comb_mask(U, unt_mat.mask_p)), apply_mask(unt_mat.rat_m, comb_mask(A, unt_mat.mask_m))])
unm_A = np.hstack([apply_mask(unt_mat.rat_p, comb_mask(A, unt_mat.mask_p)), apply_mask(unt_mat.rat_m, comb_mask(U, unt_mat.mask_m))])
unm_G = np.hstack([apply_mask(unt_mat.rat_p, comb_mask(G, unt_mat.mask_p)), apply_mask(unt_mat.rat_m, comb_mask(C, unt_mat.mask_m))])
unm_C = np.hstack([apply_mask(unt_mat.rat_p, comb_mask(C, unt_mat.mask_p)), apply_mask(unt_mat.rat_m, comb_mask(G, unt_mat.mask_m))])

In [ ]:
# plot mutation rates

fig, axs = plt.subplots(figsize=(8, 4))
sample_list = [pro_G, pro_U, pro_A, pro_C, mat_G, mat_U, mat_A, mat_C, unp_G, unp_U, unp_A, unp_C, unm_G, unm_U, unm_A, unm_C]
axs.violinplot([np.log10(i) for i in sample_list], showextrema=False)
c = 'gray'
axs.boxplot([np.log10(i) for i in sample_list], showfliers=False, widths=0.17, patch_artist=True, boxprops=dict(facecolor=c, color=c), capprops=dict(color=c), whiskerprops=dict(color=c))

plt.xticks(np.arange(1, 17, 1), ['G', 'U', 'A', 'C', 'G', 'U', 'A', 'C','G', 'U', 'A', 'C','G', 'U', 'A', 'C'])
plt.ylabel('log10(mutation rate)')
plt.show()

## Figure S1c

In [ ]:
from src.dms_utils import plot_structure, get_coords

In [ ]:
# plot DMS reactivities for high correlation regions

# 18S 21ES6cb
fig, axs = plt.subplots(2, 1, figsize=[8, 2], sharex=True)
coords = (1339, 1443)
x_18, y_nas_18, vec_norm_nas_18 = wild_nas.plot_profile(coords[0], coords[1], genome_rrna, axs[0], cmap_loc='../cmap.txt')
axs[0].set_ylim([0, 4])
axs[0].set_title('18S 21ES6cb')
axs[0].set_ylabel('DMS reactivity')
x_18, y_rib_18, vec_norm_rib_18 = wild_rib.plot_profile(coords[0], coords[1], genome_rrna, axs[1], cmap_loc='../cmap.txt')
axs[1].set_ylim([0, 10])
axs[1].set_xlabel('Position in primary rRNA transcript')
plt.show()

# 25S 63ES27a
fig, axs = plt.subplots(2, 1, figsize=[8, 2], sharex=True)
coords = (5225, 5301)
coords_tru_18 = get_coords(genome_rrna.seq[coords[0]:coords[1]], dbr_full[coords[0]:coords[1]])
x_25, y_nas_25, vec_norm_nas_25 = wild_nas.plot_profile(coords[0], coords[1], genome_rrna, axs[0], cmap_loc='../cmap.txt')
axs[0].set_ylim([0, 4])
axs[0].set_title('25S 63ES27a')
axs[0].set_ylabel('DMS reactivity')
x_25, y_rib_25, vec_norm_rib_25 = wild_rib.plot_profile(coords[0], coords[1], genome_rrna, axs[1], cmap_loc='../cmap.txt')
axs[1].set_ylim([0, 10])
axs[1].set_xlabel('Position in primary rRNA transcript')
plt.show()

In [ ]:
# plot RNA structures for high correlation regions

fig, axs = plt.subplots(2, 2, figsize=(6, 10))
fig.patch.set_facecolor('white')
axs = axs.flatten()

# 18S 21ES6cb
coords_tru_18 = get_coords(genome_rrna.seq[1339:1443], dbr_full[1339:1443])
plot_structure(coords_tru_18, genome_rrna.seq[1339:1443], y_rib_18*vec_norm_rib_18, (~genome_rrna.mask_G()&wild_nas.mask)[1339:1443], axs=axs[0], circle_size=40, text_size=6, cmap_loc='../cmap.txt')
plot_structure(coords_tru_18, genome_rrna.seq[1339:1443], y_nas_18*vec_norm_nas_18, (~genome_rrna.mask_G()&wild_nas.mask)[1339:1443], axs=axs[1], circle_size=40, text_size=6, cmap_loc='../cmap.txt')
axs[0].set_title('mature')
axs[1].set_title('nascent')

# 25S 63ES27a
coords_tru_25 = get_coords(genome_rrna.seq[5225:5301], dbr_full[5225:5301])
plot_structure(coords_tru_25, genome_rrna.seq[5225:5301], y_rib_25*vec_norm_rib_25, (~genome_rrna.mask_G()&wild_rib.mask)[5225:5301], axs=axs[2], circle_size=40, text_size=6, cmap_loc='../cmap.txt')
plot_structure(coords_tru_25, genome_rrna.seq[5225:5301], y_nas_25*vec_norm_nas_25, (~genome_rrna.mask_G()&wild_rib.mask)[5225:5301], axs=axs[3], circle_size=40, text_size=6, cmap_loc='../cmap.txt')
axs[2].set_title('mature')
axs[3].set_title('nascent')


plt.show()